In [1]:
#importing numpy  and pandas libs

import numpy as np
import pandas as pd

# importing model_selection for spliting data and other purposes

from sklearn import preprocessing, model_selection
%matplotlib inline

# we don't want warnings

import warnings
warnings.filterwarnings('ignore')

In [50]:
#loading training data from dir

df2 = pd.read_csv(filepath_or_buffer='data/finalproject/train.csv')
df = df2.copy()
df.head()

,is_duplicate,spacy_similarity,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,...,cityblock_dis,jaccard_dis,canberra_dis,euclidean_dis,minkowski_dis,braycurtis_dis,skew_q1,skew_q2,kur_q1,kur_q2
0,0,0.313089,5,36,-31,4,16,1,7,0,...,18.186985,1.0,206.461737,1.311902,1.311902,0.872352,-0.155758,-0.087513,0.188890,-0.115978
1,0,-0.067930,5,71,-66,5,24,1,12,0,...,13.787235,1.0,300.000000,1.000000,1.000000,1.000000,0.000000,0.085398,-3.000000,0.117636
2,0,0.664722,23,32,-9,12,13,4,6,1,...,11.457092,1.0,155.035952,0.849427,0.849427,0.452261,0.065101,0.079091,-0.007587,-0.470184
3,0,0.275611,19,56,-37,10,22,4,15,0,...,17.435374,1.0,203.955915,1.256324,1.256324,0.830043,-0.007600,0.018465,0.056253,0.015438
4,0,0.587580,50,30,20,21,15,10,6,0,...,14.968091,1.0,178.222529,1.086451,1.086451,0.631376,0.027209,0.037700,0.021048,-0.463481


In [3]:
# drop NA
df = df.dropna(how="any").reset_index(drop=True)

a = 0

for i in range(a, a+10):
    print(df.question1[i])
    print(df.question2[i])
    print()

What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?

What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?

How can I increase the speed of my internet connection while using a VPN?
How can Internet speed be increased by hacking through DNS?

Why am I mentally very lonely? How can I solve it?
Find the remainder when [math]23^{24}[/math] is divided by 24,23?

Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?
Which fish would survive in salt water?

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?

Should I buy tiago?
What keeps childern active and far from phone and video games?

How can I be a good geologist?
What should I do to be a great geologist?

When do you use シ instea

## Feature Engineering

In [4]:
from nltk.corpus import stopwords

#English stopwords, a, an, the, this, that ...
stop_words = stopwords.words('english')

In [5]:
question1 = "What would  a Trump presidency mean for current international master's students on an F1 visa?"
question2 = "how will a Tump presidency affect the students presently in US or planning to study in US?"

# sptliting words in each sentance

question1 = question1.lower().split()
question2 = question2.lower().split()

# checking what is the affect of stop_words on the sentance structure,
# Removing the stop_words from the quesitons

question1 = [w for w in question1 if w not in stop_words]
question2 = [w for w in question2 if w not in stop_words]

In [6]:
import gensim

from gensim.models import Word2Vec

In [7]:
# Training genism model with Google news data

model = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [8]:
# loading important libraries, we may use in the future

from pyemd import emd
from gensim.similarities import WmdSimilarity
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedLineDocument

In [9]:
#Checking how the WMD_Distance compare the two sentances
# WMD distance is how much one sentance to similar to another one

distance = model.wmdistance(question1, question2)
print('distance = %.4f' % distance)

distance = 2.3514


In [10]:
# We can compute the normalize WMD by init_sims method, 

model.init_sims(replace=True)

distance = model.wmdistance(question1, question2)

print('distance %.4f' % distance)

distance 0.9461


In [11]:
question3 = "How do we prepare for UPSC?"
question4 = "How do I prepare for civil service?"

distance = model.wmdistance(question3, question4)
print('distance = %.4f' % distance)

distance = 0.4169


In [12]:
model.wmdistance(["presently"], ["this"])

1.3247870206832886

In [13]:
question5 = "What is the stall speed and AOA of an f-14 with wings fully swept back?"
question6 = "Why did aircraft stop using variable-sweep wings, like those on an F-14?"

distance = model.wmdistance(question5, question6)
print('distance = %.4f' % distance)

distance = 0.2459


In [14]:
model.init_sims(replace=True)
distance = model.wmdistance(question5, question6)
print('distance = %.4f' % distance)

distance = 0.2459


In [15]:
# Tr out Fuzzywuzzy lib

from fuzzywuzzy import fuzz

fuzz.ratio(question1, question2)


63

In [16]:
# As we can see the fuzz lib gives us ratios of how different questions are similar to others, while it may be
# a bit harder for it to computer the ratios accuratly, it does it best to compute. We will use different lib like this one 
# to computer the ratio of all the questions and than use those features in our model

fuzz.ratio(question4, question3),fuzz.ratio(question5, question6) 


(68, 42)

In [17]:
# method for WMD distance for each question

def wmd(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    
    return model.wmdistance(q1, q2)


In [18]:
import nltk

def sent2vec(s):
    words = str(s).lower()
    words = nltk.word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    
    M = []
    
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

In [19]:
# normalize WMD distance

def norm_wmd(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    
    return norm_model.wmdistance(q1, q2)

In [20]:
# Removing stop_words, a, an, the, and, at etc

def remove_stopwords(q1):
    q1 = str(q1).lower().split()
    
    q1 = [w for w in q1 if w not in stop_words]
    
    return q1

In [21]:
# Removing stop words from each question for further processing
# the models that process the sentances when looked at similar text, the question is treated as similar even if stop words are similar
# Removing those can be benificial to us

df['question1'] = df['question1'].apply(lambda x: " ".join(remove_stopwords(x)))
df['question2'] = df['question2'].apply(lambda x:  " ".join(remove_stopwords(x)))

In [22]:
# Tokenizing, stemming, root words
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

ps = PorterStemmer() 

# Word stemming, removing tokens, confining words to its roots, programing -> program etc...
def root_words(q1):
    words = word_tokenize(q1)
    sent = [ps.stem(w) for w in words]
    return sent

In [52]:
#  changing the words to root words, this will help us recognizing the text in future from each other
# the models will be able to compare text easily even if the form of word is changed

df['question1'] = df['question1'].apply(lambda x:  " ".join(root_words(x)))
df['question2'] = df['question1'].apply(lambda x:  " ".join(root_words(x)))

In [24]:
# Removing similar words from both question for comparison

def remov_same(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    
    q1 = [w for w in q1 if ps.stem(w) not in q2]
    
    return  " ".join(q1)

In [28]:
# removing the same words, for comparison. we will have both questions with all the words and removed similar words for
# comparison purposes

df['question1_diff'] = df.apply(lambda x: remov_same(x["question1"], x["question2"]), axis=1)
df['question2_diff'] = df.apply(lambda x: remov_same(x["question2"], x["question1"]), axis=1)

In [29]:
#spaCy Similarity checking
import spacy
nlp = spacy.load('en_core_web_lg')

def spacy_sam(q1, q2):
    q1 = nlp(q1)
    q2 = nlp(q2)
    
    return q1.similarity(q2)

In [30]:
# Spacy Similarity between questions that have no same words in them

df['spacy_similarity'] = df.apply(lambda x: spacy_sam(x['question1_diff'], x['question2_diff']), axis=1)

In [53]:
# Spacy similarity between the questions with root words similar

df['spacy_sameqs_similarity'] = df.apply(lambda x: spacy_sam(x['question1'], x['question2']), axis=1)

In [31]:
#length of each string in question 1 and 2
df['len_q1'] = df['question1'].apply(lambda x: len(str(x)))
df['len_q2'] = df['question2'].apply(lambda x: len(str(x)))

# difference between question 1 and 2 lengths
df['diff_len'] = df['len_q1'] - df['len_q2']

# number of char in of each string in question 1 and 2
df['len_char_q1'] = df['question1'].apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
df['len_char_q2'] = df['question2'].apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))

# length of each word in question 1 and 2
df['len_word_q1'] = df['question1'].apply(lambda x: len(str(x).split()))
df['len_word_q2'] = df['question2'].apply(lambda x: len(str(x).split()))

# common words
df['common_words'] = df.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)

# Fuzz Ratio, the similarity or differences between each question, there are different methods in 
# fuzz class that we are using, each has its own formula of calculation

df['fuzz_ratio'] = df.apply(lambda x: fuzz.ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

df['fuzz_partial_ratio'] = df.apply(lambda x: fuzz.partial_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

df['fuzz_partial_token_set_ratio'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

df['fuzz_partial_token_sort_ratio'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

df['fuzz_token_set_ratio'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

df['fuzz_token_sort_ratio'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)


In [54]:
# fuzz ratios between question1 and question2
df['fuzz_ratio_same'] = df.apply(lambda x: fuzz.ratio(str(x['question1']), str(x['question2'])), axis=1)

df['fuzz_partial_ratio_same'] = df.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)

df['fuzz_partial_token_set_ratio_same'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)

df['fuzz_partial_token_sort_ratio_same'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

df['fuzz_token_set_ratio_same'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)

df['fuzz_token_sort_ratio_same'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

In [32]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,spacy_similarity,len_q1,len_q2,diff_len,...,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_ratio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio
0,0,1,2,india,step step guid invest share market ?,0,0.313089,5,36,-31,...,16,1,7,0,20,40,40,40,24,21
1,1,3,4,stori,would happen indian govern stole kohinoor ( ko...,0,-0.067930,5,71,-66,...,24,1,12,0,13,60,60,60,9,9
2,2,5,6,increas connect use vpn,internet speed increas hack dn ?,0,0.664722,23,32,-9,...,13,4,6,1,40,48,100,57,60,53
3,3,7,8,mental lone solv it,find remaind [ math ] 23^ { 24 } [ /math ] div...,0,0.275611,19,56,-37,...,22,4,15,0,21,32,37,42,29,27
4,4,9,10,"one dissolv quikli sugar , , methan carbon di ...",fish would surviv salt water ?,0,0.587580,50,30,20,...,15,10,6,0,38,40,39,39,32,32


In [33]:
# WMD distance between each question with same text included

df['wmd'] = df.apply(lambda x: wmd(x['question1_diff'], x['question2_diff']), axis=1)

In [55]:
#WMD distance between each question with root words, words can be same in each question

df['wmd_same'] = df.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

In [34]:
# training model with google News, for normalize distance WMD 
norm_model = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)
norm_model.init_sims(replace=True)

In [35]:
# normalize WMD disntance

df['norm_wmd'] = df.apply(lambda x: norm_wmd(x['question1_diff'], x['question2_diff']), axis=1)

In [56]:
# Normalized WMD distance for question including same words

df['norm_wmd_same'] = df.apply(lambda x: norm_wmd(x['question1'], x['question2']), axis=1)

In [36]:
from tqdm import tqdm_notebook
question1_vector = np.zeros((df.shape[0], 300))

for i, q in enumerate(tqdm_notebook(df['question1'].values)):
    question1_vector[i, :] = sent2vec(q)
    
question2_vector = np.zeros((df.shape[0], 300))

for i, q in enumerate(tqdm_notebook(df['question2'].values)):
    question2_vector[i, :] = sent2vec(q)

In [37]:
import scipy.spatial.distance as dis
from scipy.stats import skew
from scipy.stats import kurtosis 


In [38]:
# different distances between each question as features

df['cosine_dis'] = [dis.cosine(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

df['cityblock_dis'] = [dis.cityblock(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

df['jaccard_dis'] = [dis.jaccard(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

df['canberra_dis'] = [dis.canberra(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

df['euclidean_dis'] = [dis.euclidean(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

df['minkowski_dis'] = [dis.minkowski(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

df['braycurtis_dis'] = [dis.braycurtis(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

df['skew_q1'] = [skew(x) for x in np.nan_to_num(question1_vector)]

df['skew_q2'] = [skew(x) for x in np.nan_to_num(question2_vector)]

df['kur_q1'] = [kurtosis(x) for x in np.nan_to_num(question1_vector)]

df['kur_q2'] = [kurtosis(x) for x in np.nan_to_num(question2_vector)]

In [39]:
from sklearn import preprocessing


In [57]:
# removing nulls

df[df==np.inf]=np.nan
df.fillna(df.mean(), inplace=True)

In [41]:
df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'spacy_similarity', 'len_q1', 'len_q2', 'diff_len', 'len_char_q1',
       'len_char_q2', 'len_word_q1', 'len_word_q2', 'common_words',
       'fuzz_ratio', 'fuzz_partial_ratio', 'fuzz_partial_token_set_ratio',
       'fuzz_partial_token_sort_ratio', 'fuzz_token_set_ratio',
       'fuzz_token_sort_ratio', 'wmd', 'norm_wmd', 'cosine_dis',
       'cityblock_dis', 'jaccard_dis', 'canberra_dis', 'euclidean_dis',
       'minkowski_dis', 'braycurtis_dis', 'skew_q1', 'skew_q2', 'kur_q1',
       'kur_q2'],
      dtype='object')

In [58]:
#df.drop(['question1', 'question2', 'id', 'qid1', 'qid2'], axis=1, inplace=True)
df.drop(['question1', 'question2'], axis=1, inplace=True)

#df = df[pd.notnull(df['cosine_dis'])]
#df = df[pd.notnull(df['jaccard_dis'])]

X = df.loc[:, df.columns != 'is_duplicate']
y = df.loc[:, df.columns == 'is_duplicate']

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404287 entries, 0 to 404286
Data columns (total 38 columns):
is_duplicate                          404287 non-null int64
spacy_similarity                      404287 non-null float64
len_q1                                404287 non-null int64
len_q2                                404287 non-null int64
diff_len                              404287 non-null int64
len_char_q1                           404287 non-null int64
len_char_q2                           404287 non-null int64
len_word_q1                           404287 non-null int64
len_word_q2                           404287 non-null int64
common_words                          404287 non-null int64
fuzz_ratio                            404287 non-null int64
fuzz_partial_ratio                    404287 non-null int64
fuzz_partial_token_set_ratio          404287 non-null int64
fuzz_partial_token_sort_ratio         404287 non-null int64
fuzz_token_set_ratio                  404287 no

## Splitting training and test sets

In [60]:
from sklearn import preprocessing, model_selection

#splitting the dataset to training and testing sets

X_train, X_holdout, y_train, y_holdout = model_selection.train_test_split(X, y, test_size=0.2, random_state=7)

# checking the shape of our data sets

X_train.shape, X_holdout.shape, y_train.shape, y_holdout.shape

((323429, 37), (80858, 37), (323429, 1), (80858, 1))

In [47]:
df.head(10)

,is_duplicate,spacy_similarity,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,...,cityblock_dis,jaccard_dis,canberra_dis,euclidean_dis,minkowski_dis,braycurtis_dis,skew_q1,skew_q2,kur_q1,kur_q2
0,0,0.313089,5,36,-31,4,16,1,7,0,...,18.186985,1.0,206.461737,1.311902,1.311902,0.872352,-0.155758,-0.087513,0.188890,-0.115978
1,0,-0.067930,5,71,-66,5,24,1,12,0,...,13.787235,1.0,300.000000,1.000000,1.000000,1.000000,0.000000,0.085398,-3.000000,0.117636
2,0,0.664722,23,32,-9,12,13,4,6,1,...,11.457092,1.0,155.035952,0.849427,0.849427,0.452261,0.065101,0.079091,-0.007587,-0.470184
3,0,0.275611,19,56,-37,10,22,4,15,0,...,17.435374,1.0,203.955915,1.256324,1.256324,0.830043,-0.007600,0.018465,0.056253,0.015438
4,0,0.587580,50,30,20,21,15,10,6,0,...,14.968091,1.0,178.222529,1.086451,1.086451,0.631376,0.027209,0.037700,0.021048,-0.463481
5,1,0.571653,32,61,-29,15,20,7,14,0,...,17.452011,1.0,198.441203,1.251492,1.251492,0.785737,0.104864,0.013003,-0.173295,-0.556737
6,0,0.253870,9,42,-33,8,18,2,8,0,...,16.937801,1.0,199.486199,1.243934,1.243934,0.781724,-0.090366,0.042329,0.296892,-0.347765
7,1,0.628369,4,17,-13,3,10,1,3,0,...,13.644691,1.0,177.087853,0.997961,0.997961,0.588381,0.039933,0.456804,-0.165647,1.210527
8,0,0.000000,3,31,-28,2,12,2,9,0,...,14.227400,1.0,300.000000,1.000000,1.000000,1.000000,0.000000,0.205292,-3.000000,-0.409441
9,0,0.356729,31,37,-6,15,19,6,6,0,...,19.214457,1.0,216.637528,1.398589,1.398589,0.965334,-0.023789,0.169342,0.041027,-0.182079



## Training with XGBoost

In [61]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
num_round = 2
xb = xgb.XGBClassifier()
xb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [62]:
# Checkng score on test set

xb.score(X_holdout, y_holdout)

0.6918053872220435

In [63]:
X_train.fillna(X_train.mean(), inplace=True)

In [64]:
# Training with NLP Classifier

from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, ), activation='relu')
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [65]:
#MLP Scoring

mlp.score(X_holdout, y_holdout)

0.6862648099136758

In [66]:
# checking with Linear Regression

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, y_train)
clf.score(X_holdout, y_holdout)

0.6713497736773109

### The XGBoost score was higher so we will tune that model only, we don't need other models for now


In [67]:

from sklearn.model_selection import GridSearchCV, StratifiedKFold
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.02], #so called `eta` value
              'max_depth': np.arange(4, 10),
              'min_child_weight': np.arange(7, 10),
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.5],
              'n_estimators': [500]}
xgb_tunned = GridSearchCV(xgb.XGBClassifier(), parameters, n_jobs=-1, verbose=2, refit=True)

In [68]:
%%time
xgb_tunned.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 39.6min
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed: 87.3min finished


Wall time: 1h 30min 12s


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'nthread': [4], 'objective': ['binary:logistic'], 'learning_rate': [0.02], 'max_depth': array([4, 5, 6, 7, 8, 9]), 'min_child_weight': array([7, 8, 9]), 'silent': [1], 'subsample': [0.8], 'colsample_bytree': [0.5], 'n_estimators': [500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [69]:
xgb_tunned.score(X_holdout, y_holdout)

0.7125825521284227

In [73]:
df2.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## checking that feature engineering on original test set

In [70]:
dftest = pd.read_csv(filepath_or_buffer='data/finalproject/test.csv')

In [ ]:
# drop NA
dftest = dftest.dropna(how="any").reset_index(drop=True)

# Removing stop words from each question for further processing
# the models that process the sentances when looked at similar text, the question is treated as similar even if stop words are similar
# Removing those can be benificial to us

dftest['question1'] = dftest['question1'].apply(lambda x: " ".join(remove_stopwords(x)))
dftest['question2'] = dftest['question2'].apply(lambda x:  " ".join(remove_stopwords(x)))

#  changing the words to root words, this will help us recognizing the text in future from each other
# the models will be able to compare text easily even if the form of word is changed

dftest['question1'] = dftest['question1'].apply(lambda x:  " ".join(root_words(x)))
dftest['question2'] = dftest['question1'].apply(lambda x:  " ".join(root_words(x)))

# removing the same words, for comparison. we will have both questions with all the words and removed similar words for
# comparison purposes

dftest['question1_diff'] = dftest.apply(lambda x: remov_same(x["question1"], x["question2"]), axis=1)
dftest['question2_diff'] = dftest.apply(lambda x: remov_same(x["question2"], x["question1"]), axis=1)

In [ ]:
# Spacy Similarity between questions that have no same words in them

dftest['spacy_similarity'] = dftest.apply(lambda x: spacy_sam(x['question1_diff'], x['question2_diff']), axis=1)

# Spacy similarity between the questions with root words similar

dftest['spacy_sameqs_similarity'] = dftest.apply(lambda x: spacy_sam(x['question1'], x['question2']), axis=1)

In [ ]:
#length of each string in question 1 and 2
dftest['len_q1'] = dftest['question1'].apply(lambda x: len(str(x)))
dftest['len_q2'] = dftest['question2'].apply(lambda x: len(str(x)))

# difference between question 1 and 2 lengths
dftest['diff_len'] = dftest['len_q1'] - dftest['len_q2']

# number of char in of each string in question 1 and 2
dftest['len_char_q1'] = dftest['question1'].apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
dftest['len_char_q2'] = dftest['question2'].apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))

# length of each word in question 1 and 2
dftest['len_word_q1'] = dftest['question1'].apply(lambda x: len(str(x).split()))
dftest['len_word_q2'] = dftest['question2'].apply(lambda x: len(str(x).split()))

# common words
dftest['common_words'] = dftest.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)

# Fuzz Ratio, the similarity or differences between each question, there are different methods in 
# fuzz class that we are using, each has its own formula of calculation

dftest['fuzz_ratio'] = dftest.apply(lambda x: fuzz.ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

dftest['fuzz_partial_ratio'] = dftest.apply(lambda x: fuzz.partial_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

dftest['fuzz_partial_token_set_ratio'] = dftest.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

dftest['fuzz_partial_token_sort_ratio'] = dftest.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

dftest['fuzz_token_set_ratio'] = dftest.apply(lambda x: fuzz.token_set_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)

dftest['fuzz_token_sort_ratio'] = dftest.apply(lambda x: fuzz.token_sort_ratio(str(x['question1_diff']), str(x['question2_diff'])), axis=1)


In [ ]:
# fuzz ratios between question1 and question2
dftest['fuzz_ratio_same'] = dftest.apply(lambda x: fuzz.ratio(str(x['question1']), str(x['question2'])), axis=1)

dftest['fuzz_partial_ratio_same'] = dftest.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)

dftest['fuzz_partial_token_set_ratio_same'] = dftest.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)

dftest['fuzz_partial_token_sort_ratio_same'] = dftest.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

dftest['fuzz_token_set_ratio_same'] = dftest.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)

dftest['fuzz_token_sort_ratio_same'] = dftest.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

In [ ]:
# WMD distance between each question with same text included

dftest['wmd'] = dftest.apply(lambda x: wmd(x['question1_diff'], x['question2_diff']), axis=1)

#WMD distance between each question with root words, words can be same in each question

dftest['wmd_same'] = dftest.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

# normalize WMD disntance

dftest['norm_wmd'] = dftest.apply(lambda x: norm_wmd(x['question1_diff'], x['question2_diff']), axis=1)

# Normalized WMD distance for question including same words

dftest['norm_wmd_same'] = dftest.apply(lambda x: norm_wmd(x['question1'], x['question2']), axis=1)

In [ ]:
question1_vector = np.zeros((dftest.shape[0], 300))

for i, q in enumerate(tqdm_notebook(dftest['question1'].values)):
    question1_vector[i, :] = sent2vec(q)
    
question2_vector = np.zeros((dftest.shape[0], 300))

for i, q in enumerate(tqdm_notebook(dftest['question2'].values)):
    question2_vector[i, :] = sent2vec(q)

In [ ]:
# different distances between each question as features

dftest['cosine_dis'] = [dis.cosine(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

dftest['cityblock_dis'] = [dis.cityblock(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

dftest['jaccard_dis'] = [dis.jaccard(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

dftest['canberra_dis'] = [dis.canberra(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

dftest['euclidean_dis'] = [dis.euclidean(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

dftest['minkowski_dis'] = [dis.minkowski(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

dftest['braycurtis_dis'] = [dis.braycurtis(x,y) for (x, y) in zip(np.nan_to_num(question1_vector), np.nan_to_num(question2_vector))]

dftest['skew_q1'] = [skew(x) for x in np.nan_to_num(question1_vector)]

dftest['skew_q2'] = [skew(x) for x in np.nan_to_num(question2_vector)]

dftest['kur_q1'] = [kurtosis(x) for x in np.nan_to_num(question1_vector)]

dftest['kur_q2'] = [kurtosis(x) for x in np.nan_to_num(question2_vector)]